# **1.Set up**

In [ ]:
import random
import os
import cv2
import numpy as np
import PIL

from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from skimage import feature


from google.colab import drive
drive.mount('/content/drive')

!pwd
%cd /content/drive/MyDrive/10701 Project/Places365
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
/content/drive/.shortcut-targets-by-id/1LctJnKZFNvssVRAeqcgI-q6k_UArdIHZ/Places365
categories_places365.txt	 train_256_places365standard.tar
data_256_standard		 train_hog_images_20k_to_40k.npy
filelist_places365-standard.tar  train_hog_images.npy
places365_test.txt		 train_hog_labels_20k_to_40k.npy
places365_train_standard.txt	 train_hog_labels.npy
test_hog_images.npy		 val_256
test_hog_labels.npy		 val_256.tar


In [ ]:
with open('places365_train_standard.txt') as f:
    lines = f.readlines()

sample_n = len(lines)
random.shuffle(lines)

print("Loaded", sample_n, "samples")

train_n = sample_n // 10 * 7
val_n = sample_n // 10 * 2
test_n = sample_n - train_n - val_n

train_entries = lines[ : train_n]
val_entries = lines[train_n : train_n+val_n]
test_entries = lines[train_n+val_n:]


Loaded 149168 samples


In [ ]:
# This function splits every entry and returns (paths, labels)
# /a/airfield/00000001.jpg 0
def split_entries(entries):
  paths = []
  labels = []
  
  for entry in entries:
    parts = entry.split(' ')
    paths.append(parts[0])
    labels.append(parts[1])
  
  return (paths, labels)

In [ ]:
(train_paths, train_labels) = split_entries(train_entries)
(val_paths, val_labels) = split_entries(val_entries)
(test_paths, test_labels) = split_entries(test_entries)

# Images = []
# Labels = []

In [ ]:
Images = []
Labels = []

# **Old Code**

In [ ]:
def hog(paths, labels):
  N = len(paths)

  for i in range(N):
    # prepare file path and label
    path = paths[i]
    # print('i=', i, 'path', path)
    image_path = 'data_256_standard' + path
    label = int(labels[i][:-1])

    # read image
    image = np.asarray(PIL.Image.open(image_path))
    image = cv2.resize(image, (128, 256))

    # hog
    hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')
    
    # update loop variable
    Images.append(hog_desc)
    Labels.append(label)
    


In [ ]:
# hog(train_paths[:3000], train_labels[:3000])
# print(Images)
# print(Labels)

# **Load and Append the hog features and labels of the train images**

In [ ]:
# Load first 20,000 train images
final_train_images = np.load('train_hog_images.npy')
final_train_labels = np.load('train_hog_labels.npy')



In [ ]:
def train_initialize(trainImages, trainLabels):
  N = len(trainImages)

  for i in range(N):
    hog_feature = trainImages[i]
    label = trainLabels[i]

    Images.append(hog_feature)
    Labels.append(label)

In [ ]:
# Append the first 20,000 train images into the final lists
train_initialize(final_train_images, final_train_labels)


# **Train SVM**

In [ ]:
print('Training on train images...')
# svm_model = LinearSVC(random_state=42, tol=1e-5)
svm_model = LinearSVC(dual=False, tol=1e-3, C=0.5)
svm_model.fit(Images, Labels)
print('Finish training SVM')


Training on train images...
Finish training SVM


#Save Result#

SVC: \\
n = 1000: 0.2 \\
`[:2000]` 0.186 \\
`[2000:4000]` 0.203 \\

n = 10000: 0.3 \\
`[:50]` 0.28

LinearSVC: \\
n = 1000: 0.17 \\
n = 3000: 0.192 \\


# **Extract the hog features and labels of the test dataset**

In [ ]:
def test_extract(paths, labels):
  N = len(paths)

  for i in range(N):
    # prepare file path and label
    path = paths[i]
    image_path = 'data_256_standard' + path
    label = int(labels[i][:-1])

    # read image
    image = np.asarray(PIL.Image.open(image_path))
    image = cv2.resize(image, (128, 256))

    # hog
    hog_desc = feature.hog(image, orientations=9, pixels_per_cell=(8, 8),
            cells_per_block=(2, 2), transform_sqrt=True, block_norm='L2-Hys')

    TestImages.append(hog_desc)
    TestLabels.append(label)


In [ ]:
# TestImages = []
# TestLabels = []
# test_extract(test_paths, test_labels)
# print(TestImages)
# print(TestLabels)


In [ ]:
# np.save('test_hog_images.npy', TestImages)
# np.save('test_hog_labels.npy', TestLabels)

In [ ]:
# tmp_labels = np.load('test_hog_labels.npy')
# print(TestLabels)
# print()
# print(list(tmp_labels))

In [ ]:
# tmp_images = np.load('test_hog_images.npy')
# print(TestImages)
# print()
# print(list(tmp_images))
# print(TestImages == tmp_images)

In [ ]:
final_test_images = np.load('test_hog_images.npy')
final_test_labels = np.load('test_hog_labels.npy')

In [ ]:
def test_accuracy(images, labels):
  N = len(images)
  correctN = 0

  for i in range(N):
    hog_feature = final_test_images[i]
    label = labels[i]
    
    pred = svm_model.predict(hog_feature.reshape(1, -1))[0]

    if pred == label:
      correctN += 1
    
  return correctN /N

In [ ]:
def train_accuracy(images, labels):
  N = len(images)
  correctN = 0

  for i in range(N):
    hog_feature = images[i]
    label = labels[i]

    pred = svm_model.predict(hog_feature.reshape(1, -1))[0]

    if pred == label:
      correctN += 1

  return correctN /N

# **Run this after you train your SVM**

In [ ]:
train_accuracy = train_accuracy(final_train_images, final_train_labels)
print("n = 20000 accuracy", train_accuracy)

n = 20000 accuracy 0.9999


In [ ]:
test_accuracy = test_accuracy(final_test_images, final_test_labels)
print("n = 20000 accuracy", test_accuracy)

n = 20000 accuracy 0.2908737603859555
